In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
from aif360.algorithms.preprocessing import Reweighing
from sklearn.base import BaseEstimator, ClassifierMixin
from aif360.sklearn.inprocessing import ExponentiatedGradientReduction
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import LinearSVC
from sklearn.metrics import accuracy_score, recall_score, f1_score

/Users/paolocarminevalletta/Documents/SE4AI/Fairness_datasmell_SE4AI/ambiente/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [4]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset, predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attributes):
    features = [privileged_attribute] + unprivileged_attributes

    aif_race_dataset = BinaryLabelDataset(
            df=dataset,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    aif_race_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    race_privileged_group = [{privileged_attribute: 1}]
    race_unprivileged_groups = [{attr: 1} for attr in unprivileged_attributes]

    fairness_metrics = ClassificationMetric(dataset=aif_race_dataset,
                            classified_dataset=aif_race_pred,
                            unprivileged_groups=race_unprivileged_groups,
                            privileged_groups=race_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Average Odds Difference (AOD): {AOD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")

In [5]:
dataset_path = 'adult/rawAdult-trained.csv'
df_raw = pd.read_csv(dataset_path)
pd.set_option('display.max_columns', None)
df_raw.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,sex_Female
0,39,77516.0,13.0,2174.0,0.0,40.0,0,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,50,83311.0,13.0,0.0,0.0,13.0,0,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,38,215646.0,9.0,0.0,0.0,40.0,0,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [6]:
X = df_raw.drop(columns="income")
y = df_raw["income"]

privileged_attributeSex = "sex_Male"
unprivileged_attributesSex = ["sex_Female"]
sex_features = [privileged_attributeSex] + unprivileged_attributesSex

privileged_attributeRace = "race_White"
unprivileged_attributesRace = ["race_Black", "race_Other", "race_Asian-Pac-Islander"]
race_features = [privileged_attributeRace] + unprivileged_attributesRace

In [7]:
# Define four sets and apply the function
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # 0.2 indicates a test set size of 20%
                                                    random_state=42)

In [8]:
dt_clf = DecisionTreeClassifier(random_state=42)

# The fit function will do the trick
dt_clf.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
dt_predictions = dt_clf.predict(X_test)

compute_performance_metrics(y_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.8174838775719111
Decision Tree Recall: 0.619108280254777
Decision Tree F1 Score: 0.6205575654394552


In [9]:
svm_pipeline = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))

# Addestra il classificatore sui dati di training 
svm_classifier = svm_pipeline.named_steps['linearsvc'].fit(X_train, y_train.values.ravel())

# Fai delle predizioni sui dati di test
svm_pred = svm_classifier.predict(X_test)

compute_performance_metrics(y_test,svm_pred,"SVM")

SVM Accuracy: 0.8220902855972976
SVM Recall: 0.3813163481953291
SVM F1 Score: 0.5082059988681381


In [11]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.8537209540382844
Random Forest Recall: 0.6084925690021231
Random Forest F1 Score: 0.6672875436554133


In [12]:
# We create an ExponentiatedGradientReduction object that acts on the trained model to change its internal behavior
processed_dt_clf = ExponentiatedGradientReduction(prot_attr=sex_features, estimator=dt_clf, constraints='DemographicParity', drop_prot_attr=False)
processed_svm_classifier = ExponentiatedGradientReduction(prot_attr=sex_features, estimator=svm_classifier, constraints='DemographicParity', drop_prot_attr=False)
processed_rf_classifier = ExponentiatedGradientReduction(prot_attr=sex_features, estimator=rf_classifier, constraints='DemographicParity', drop_prot_attr=False)

In [ ]:
# Then we train again the same model updated with the reduction
fair_dt_clf = processed_dt_clf.fit(X_train, y_train)
fair_svm_classifier = processed_svm_classifier.fit(X_train, y_train)
fair_rf_classifier = processed_rf_classifier.fit(X_train, y_train)

In [14]:
fair_dt_predictions = fair_dt_clf.predict(X_test)
fair_svm_predictions = fair_svm_classifier.predict(X_test)
fair_rf_predictions = fair_rf_classifier.predict(X_test)

In [15]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_test.copy(deep=True) # we create a copy of the test set
dataset['income'] = y_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['income'] = fair_dt_predictions # but this time the target feature is made by the predictions of our model

In [18]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.011
Average Odds Difference (AOD): 0.054
Equal Opportunity Difference (EOD): 0.022


In [19]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L'SVM
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
predictions = dataset.copy(deep=True) 
predictions['income'] = fair_svm_predictions

In [20]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.008
Average Odds Difference (AOD): 0.148
Equal Opportunity Difference (EOD): 0.262


In [21]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
predictions = dataset.copy(deep=True) 
predictions['income'] = fair_rf_predictions

In [22]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.013
Average Odds Difference (AOD): 0.025
Equal Opportunity Difference (EOD): -0.056


LAVORO SU RACE

In [23]:
dt_clf = DecisionTreeClassifier(random_state=42)

# The fit function will do the trick
dt_clf.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
dt_predictions = dt_clf.predict(X_test)

compute_performance_metrics(y_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.8174838775719111
Decision Tree Recall: 0.619108280254777
Decision Tree F1 Score: 0.6205575654394552


In [24]:
# Crea la pipeline senza addestrarla
svm_pipeline = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))

# Addestra il classificatore sui dati di training 
svm_classifier = svm_pipeline.named_steps['linearsvc'].fit(X_train, y_train.values.ravel())

# Fai delle predizioni sui dati di test
svm_pred = svm_classifier.predict(X_test)

compute_performance_metrics(y_test,svm_pred,"SVM")

SVM Accuracy: 0.8220902855972976
SVM Recall: 0.3813163481953291
SVM F1 Score: 0.5082059988681381


In [25]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.8537209540382844
Random Forest Recall: 0.6084925690021231
Random Forest F1 Score: 0.6672875436554133


In [26]:
# We create an ExponentiatedGradientReduction object that acts on the trained model to change its internal behavior
processed_dt_clf = ExponentiatedGradientReduction(prot_attr=race_features, estimator=dt_clf, constraints='DemographicParity', drop_prot_attr=False)
processed_svm_classifier = ExponentiatedGradientReduction(prot_attr=race_features, estimator=svm_classifier, constraints='DemographicParity', drop_prot_attr=False)
processed_rf_classifier = ExponentiatedGradientReduction(prot_attr=race_features, estimator=rf_classifier, constraints='DemographicParity', drop_prot_attr=False)

In [ ]:
# Then we train again the same model updated with the reduction
fair_dt_clf = processed_dt_clf.fit(X_train, y_train)
fair_svm_classifier = processed_svm_classifier.fit(X_train, y_train)
fair_rf_classifier = processed_rf_classifier.fit(X_train, y_train)

In [28]:
fair_dt_predictions = fair_dt_clf.predict(X_test)
fair_svm_predictions = fair_svm_classifier.predict(X_test)
fair_rf_predictions = fair_rf_classifier.predict(X_test)

In [29]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_test.copy(deep=True) # we create a copy of the test set
dataset['income'] = y_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['income'] = fair_dt_predictions # but this time the target feature is made by the predictions of our model

In [30]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.023
Average Odds Difference (AOD): 0.01
Equal Opportunity Difference (EOD): -0.007


In [31]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L'SVM
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
predictions = dataset.copy(deep=True) 
predictions['income'] = fair_svm_predictions

In [32]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.01
Average Odds Difference (AOD): 0.068
Equal Opportunity Difference (EOD): 0.093


In [33]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
predictions = dataset.copy(deep=True) 
predictions['income'] = fair_rf_predictions

In [34]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): 0.025
Average Odds Difference (AOD): 0.047
Equal Opportunity Difference (EOD): 0.004
